In [ ]:
import torchvision
from model import res2net18
import os
import parameters
import function
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from tqdm import tqdm


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))
    epochs = parameters.epoch
    save_model = parameters.resnet_save_model
    save_path = parameters.resnet_save_path_CIFAR100

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(32),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),

        "val": transforms.Compose([transforms.Resize((32, 32)),  # cannot 224, must (224, 224)
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
    }

    train_dataset = torchvision.datasets.CIFAR100(root='./data/CIFAR100', train=True,
                                                  download=True, transform=data_transform["train"])

    val_dataset = torchvision.datasets.CIFAR100(root='./data/CIFAR100', train=False,
                                                download=False, transform=data_transform["val"])

    train_num = len(train_dataset)
    val_num = len(val_dataset)
    print("using {} images for training, {} images for validation.".format(train_num, val_num))
    # #################################################################################################################

    batch_size = parameters.batch_size

    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    # ##################################################################################################################
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               pin_memory=True,
                                               num_workers=nw,
                                               )

    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             pin_memory=True,
                                             num_workers=nw,
                                             )

    model = res2net18()
    model.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=parameters.resnet_lr)
    best_acc = 0.0

    # 为后面制作表图
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []

    for epoch in range(epochs):
        model.train()
        running_loss_train = 0.0
        train_accurate = 0.0
        train_bar = tqdm(train_loader)
        for images, labels in train_bar:
            optimizer.zero_grad()

            outputs = model(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            predict = torch.max(outputs, dim=1)[1]
            train_accurate += torch.eq(predict, labels.to(device)).sum().item()
            running_loss_train += loss.item()

        train_accurate = train_accurate / train_num
        running_loss_train = running_loss_train / train_num
        train_acc_list.append(train_accurate)
        train_loss_list.append(running_loss_train)

        print('[epoch %d] train_loss: %.7f  train_accuracy: %.3f' %
              (epoch + 1, running_loss_train, train_accurate))

        # validate
        model.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_loader = tqdm(val_loader)
            for val_data in val_loader:
                val_images, val_labels = val_data
                outputs = model(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        val_acc_list.append(val_accurate)
        print('[epoch %d] val_accuracy: %.3f' %
              (epoch + 1, val_accurate))

        function.writer_into_excel_onlyval(save_path, train_loss_list, train_acc_list, val_acc_list, "CIFAR100")

        # 选择最best的模型进行保存 评价指标此处是acc
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(model.state_dict(), save_model)


if __name__ == '__main__':
    main()

using cuda:0 device.
Files already downloaded and verified
using 50000 images for training, 10000 images for validation.
Using 2 dataloader workers every process


100%|██████████| 6250/6250 [03:39<00:00, 28.42it/s]


[epoch 1] train_loss: 0.5106577  train_accuracy: 0.077


100%|██████████| 1250/1250 [00:17<00:00, 71.36it/s]


[epoch 1] val_accuracy: 0.139
save success!   .


...

[epoch 9] val_accuracy: 0.432
save success!   .


100%|██████████| 6250/6250 [03:27<00:00, 30.12it/s]


[epoch 10] train_loss: 0.3051982  train_accuracy: 0.378


100%|██████████| 1250/1250 [00:15<00:00, 79.19it/s]


[epoch 10] val_accuracy: 0.451
save success!   .


In [40]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import sys

sys.path.append('/content/drive/MyDrive/Colab Notebooks')
batch_size = 8
# lr=0.002
lr = 0.001
momentum = 0.9
epochs = 20
print_every=2000

DATASET_PATH='./input'
RESULT_FOLDER='./Results'
if not os.path.exists(RESULT_FOLDER):
    # 使用os.makedirs()函数创建文件夹
    os.makedirs(RESULT_FOLDER)
resFilename=RESULT_FOLDER+'res2net-10.txt'

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)
# 1.load data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# 这是已经划分好训练和测试数据集了的
trainset = torchvision.datasets.CIFAR100(root=DATASET_PATH, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root=DATASET_PATH, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = {19: 'cattle', 29: 'dinosaur', 0: 'apple', 11: 'boy', 1: 'aquarium_fish', 86: 'telephone',
           90: 'train', 28: 'cup', 23: 'cloud', 31: 'elephant', 39: 'keyboard', 96: 'willow_tree',
           82: 'sunflower', 17: 'castle', 71: 'sea', 8: 'bicycle', 97: 'wolf', 80: 'squirrel',
           74: 'shrew', 59: 'pine_tree', 70: 'rose', 87: 'television', 84: 'table', 64: 'possum',
           52: 'oak_tree', 42: 'leopard', 47: 'maple_tree', 65: 'rabbit', 21: 'chimpanzee',
           22: 'clock', 81: 'streetcar', 24: 'cockroach', 78: 'snake', 45: 'lobster', 49: 'mountain',
           56: 'palm_tree', 76: 'skyscraper', 89: 'tractor', 73: 'shark', 14: 'butterfly', 9: 'bottle',
           6: 'bee', 20: 'chair', 98: 'woman', 36: 'hamster', 55: 'otter', 72: 'seal', 43: 'lion', 51: 'mushroom',
           35: 'girl', 83: 'sweet_pepper', 33: 'forest', 27: 'crocodile', 53: 'orange', 92: 'tulip', 50: 'mouse',
           15: 'camel', 18: 'caterpillar', 46: 'man', 75: 'skunk', 38: 'kangaroo', 66: 'raccoon', 77: 'snail',
           69: 'rocket', 95: 'whale', 99: 'worm', 93: 'turtle', 4: 'beaver', 61: 'plate', 94: 'wardrobe', 68: 'road',
           34: 'fox', 32: 'flatfish', 88: 'tiger', 67: 'ray', 30: 'dolphin', 62: 'poppy', 63: 'porcupine', 40: 'lamp',
           26: 'crab', 48: 'motorcycle', 79: 'spider', 85: 'tank', 54: 'orchid', 44: 'lizard', 7: 'beetle',
           12: 'bridge',
           2: 'baby', 41: 'lawn_mower', 37: 'house', 13: 'bus', 25: 'couch', 10: 'bowl', 57: 'pear', 5: 'bed',
           60: 'plain', 91: 'trout', 3: 'bear', 58: 'pickup_truck', 16: 'can'}

#Note that the org code if from https://github.com/houqb/CoordAttention/blob/main/coordatt.py
#SE模块
class SEModule(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        x = self.avg_pool(input)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return input * x

class Bottle2neck(nn.Module):
    
    expansion = 4  #残差块的输出通道数=输入通道数*expansion

    def __init__(self, inplanes, planes, downsample=None, stride=1, scale=4, groups=1, se=False,  norm_layer=True):
        #scales为残差块中使用分层的特征组数，groups表示其中3*3卷积层数量，SE模块和BN层
        super(Bottle2neck, self).__init__()

        if planes % scale != 0: #输出通道数为4的倍数
            raise ValueError('Planes must be divisible by scale')
        if norm_layer:  #BN层
            norm_layer = nn.BatchNorm2d

        bottleneck_planes = groups * planes
        self.scale = scale
        self.stride = stride
        self.downsample = downsample

        #1*1的卷积层,在第二个layer时缩小图片尺寸
        self.conv1 = nn.Conv2d(inplanes, bottleneck_planes, kernel_size=1, stride=stride)
        self.bn1 = norm_layer(bottleneck_planes)
        
        #3*3的卷积层，一共有3个卷积层和3个BN层
        self.conv2 = nn.ModuleList([nn.Conv2d(bottleneck_planes // scale, bottleneck_planes // scale,
                                      kernel_size=3, stride=1, padding=1, groups=groups) for i in range(scale-1)])
        self.bn2 = nn.ModuleList([norm_layer(bottleneck_planes // scale) for _ in range(scale-1)])
        
        #1*1的卷积层，经过这个卷积层之后输出的通道数变成
        self.conv3 = nn.Conv2d(bottleneck_planes, planes * self.expansion, kernel_size=1, stride=1)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        #SE模块
        self.se = SEModule(planes * self.expansion) if se else None

    def forward(self, x):

        identity = x

        #1*1的卷积层
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        #scales个(3x3)的残差分层架构
        xs = torch.chunk(out, self.scale, 1) #将x分割成scales块
        ys = []
        for s in range(self.scale):
            if s == 0:
                ys.append(xs[s])
            elif s == 1:
                ys.append(self.relu(self.bn2[s-1](self.conv2[s-1](xs[s]))))
            else:
                ys.append(self.relu(self.bn2[s-1](self.conv2[s-1](xs[s] + ys[-1]))))
        out = torch.cat(ys, 1)

        #1*1的卷积层
        out = self.conv3(out)
        out = self.bn3(out)

        #加入SE模块
        if self.se is not None:
            out = self.se(out)
        #下采样
        if self.downsample:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)

        return out

class Res2Net(nn.Module):
    def __init__(self, layers, num_classes=100, width=16, scale=4, groups=1,
                 zero_init_residual=True, se=False, norm_layer=True):
        super(Res2Net, self).__init__()
        if norm_layer:  #BN层
            norm_layer = nn.BatchNorm2d
        #通道数分别为64,128,256,512
        self.inplanes = 64
        self.scale = scale
        #7*7的卷积层，3*3的最大池化层:改为3*3的卷积
        self.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        #四个残差块
        self.layer1 = self._make_layer(Bottle2neck,64,layers[0],stride=1,scale=scale,groups=groups, se=se, norm_layer=norm_layer)
        self.layer2 = self._make_layer(Bottle2neck,128,layers[1],stride=2,scale=scale,groups=groups, se=se, norm_layer=norm_layer)
        self.layer3 = self._make_layer(Bottle2neck,256,layers[2],stride=2,scale=scale,groups=groups, se=se, norm_layer=norm_layer)
        self.layer4 = self._make_layer(Bottle2neck,512,layers[3],stride=2,scale=scale,groups=groups, se=se, norm_layer=norm_layer)
        #自适应平均池化，全连接层
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * Bottle2neck.expansion, num_classes)

        #初始化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        # #零初始化每个剩余分支中的最后一个BN，以便剩余分支从零开始，并且每个剩余块的行为类似于一个恒等式
        # if zero_init_residual:
        #     for m in self.modules():
        #         if isinstance(m, Bottle2neck):
        #             nn.init.constant_(m.bn3.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, scale=4, groups=1, se=False, norm_layer=True):
        # if norm_layer:
        #     norm_layer = nn.BatchNorm2d
        downsample = None 
        #下采样，可缩小图片尺寸
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, 
                     kernel_size=1, stride=stride),nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, downsample, stride=stride, scale=scale, groups=groups, se=se, norm_layer=norm_layer))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, scale=scale, groups=groups, se=se, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def res2net18():
    """Constructs a Res2Net-18_v1b model.
    Res2Net-18 refers to the Res2Net-50_v1b_26w_4s.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = Res2Net(layers=[2, 2, 2, 2])
    return model


net=res2net18().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

# 开始训练
loss_vector, accuracy_vector = [], []
train_loss_vector = []
train_accuracy_vector = []
with open(resFilename, "w") as f:
    for epoch in range(epochs):
        # 训练
        net.train()
        running_loss = 0.0
        total = 0
        all = 0
        running_loss = 0.0
        train_loss = 0.0
        train_accuracy = 0.0
        correct = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            all += labels.size(0)
            correct += predicted.eq(labels.data).sum()
            train_accuracy += predicted.eq(labels.data).cpu().sum()
            if i % print_every == print_every-1:  # print every 2000 mini-batches
                print(
                    f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f},Train accuracy:{100 * correct / total:.2f}')
                f.write('[%03d  %05d] |Loss: %.03f |Train accuracy: %.02f  '
                        % (epoch + 1, i + 1, running_loss / 2000, 100 * correct / total))
                f.write('\n')
                f.flush()
                total = 0;
                running_loss = 0.0
                correct = 0
        train_loss /= all
        train_accuracy = 100 * train_accuracy / all
        train_loss_vector.append(train_loss)
        train_accuracy_vector.append(train_accuracy)

        # 每训练完一个epoch测试一下准确率
        # 测试不需要反向计算梯度：提高效率
        with torch.no_grad():
            net.eval()
            total = 0
            val_loss, correct = 0, 0
            for data, target in testloader:
                data = data.to(device)
                target = target.to(device)
                output = net(data)
                val_loss += criterion(output, target).data.item()
                _, predicted = torch.max(output.data, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)  # total就是一个 总数...

            val_loss /= total
            loss_vector.append(val_loss)

            accuracy = 100 * correct / total
            accuracy_vector.append(accuracy)

            print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
                val_loss, correct, total, accuracy))
            f.write('\nValidation set: Average loss:%4f,Accuracy:%d/%d = %2f \n'
                    % (val_loss, correct, total, accuracy))
            f.flush()

print("training finished！")
plt.figure(figsize=(5, 3))
plt.plot(np.arange(1, epochs + 1), loss_vector, label='Vali_Loss')
plt.plot(np.arange(1, epochs + 1), train_loss_vector, label='Train_Loss')
plt.title('loss,epoch=%s' % epochs)
# 显示图例
plt.legend()
plt.show()

plt.figure(figsize=(5, 3))
plt.plot(np.arange(1, epochs + 1), accuracy_vector, label='Vali_accura')
plt.plot(np.arange(1, epochs + 1), train_accuracy_vector, label='train_accura')
plt.title('accuracy,epoch=%s' % epochs)
plt.legend()
plt.show()

Using PyTorch version: 2.0.1+cu118  Device: cuda
Files already downloaded and verified
Files already downloaded and verified


RuntimeError: ignored

In [ ]:
!pip install xlwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.empty_cache()